In [ ]:
#Evaluation of an essay using parallel workflows
#evealuating language, analysis and clarity in parallel and then combining the results
#each evaluation returns a score and feedback
#final evaluation combines the feedbacks and averages the scores
#we are using reducer to aggregate the scores from parallel evaluations

from langgraph.graph import StateGraph,START,END
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from typing import TypedDict,Annotated
from pydantic import BaseModel, Field
load_dotenv()
model=ChatOpenAI(model="gpt-4o-mini")
import operator # to use the reducer function


In [35]:
# define the schema for strucured output
class EvaluationSchema(BaseModel): 
    feedback:str=Field(description="Detailed feedback on the essay provided.")
    score:int=Field(description="score out of 10",ge=0,le=10)


In [36]:
#defining the structured model
Structured_model=model.with_structured_output(EvaluationSchema)

In [37]:
essay="""" Black holes are among the most fascinating and enigmatic phenomena in the universe. Predicted by Einstein’s theory of general relativity and confirmed through modern observations, black holes challenge our understanding of space, time, and the fundamental laws of physics. Though invisible by nature, their powerful influence on surrounding matter makes them some of the most studied objects in modern astrophysics.

What Is a Black Hole?

A black hole is a region of space where gravity is so strong that nothing—not even light—can escape from it. This immense gravitational pull arises when a large amount of mass is compressed into an extremely small area. At the center of a black hole lies the singularity, a point where density becomes infinite and the known laws of physics break down. Surrounding the singularity is the event horizon, the boundary beyond which escape is impossible.

How Do Black Holes Form?

Most black holes form from the remnants of massive stars. When such a star exhausts its nuclear fuel, it can no longer support itself against gravity and collapses inward, triggering a supernova explosion. If the remaining core is sufficiently massive, it collapses completely, forming a black hole. In addition to these stellar-mass black holes, scientists have discovered supermassive black holes—millions or even billions of times more massive than the Sun—at the centers of most galaxies."""

In [38]:
'''prompt=f'Evaluate the language quality of the following essay and give it a score out of 10:\n\n{essay}'
result=Structured_model.invoke(prompt)
print(result.score)'''

"prompt=f'Evaluate the language quality of the following essay and give it a score out of 10:\n\n{essay}'\nresult=Structured_model.invoke(prompt)\nprint(result.score)"

In [39]:
#define the state now
class UPSCState(TypedDict):
    essay:str
    language_feedback:str
    analysis_feedback:str
    clarity_feedback:str
    overall_feedback:str
    individual_scores:Annotated[list[int],operator.add]
    average_score:float
    

In [40]:
def evaluate_language(state:UPSCState)->UPSCState:
    prompt=f'Evaluate the language quality of the following essay and give it a score out of 10:\n\n{state["essay"]}'
    output=Structured_model.invoke(prompt)

    return {'language_feedback':output.feedback,'individual_scores':[output.score]}

In [41]:
def evaluate_analysis(state:UPSCState)->UPSCState:
    prompt=f'Evaluate the depth of analysis of the following essay and give it a score out of 10:\n\n{state["essay"]}'
    output=Structured_model.invoke(prompt)

    return {'analysis_feedback':output.feedback,'individual_scores':[output.score]}

In [42]:
def evaluate_thought(state:UPSCState)->UPSCState:
    prompt=f'Evaluate the clarity of thought in the following essay and give it a score out of 10:\n\n{state["essay"]}'
    output=Structured_model.invoke(prompt)

    return {'clarity_feedback':output.feedback,'individual_scores':[output.score]}

In [43]:
def final_evaluation(state:UPSCState)->UPSCState:
    #summary feedback
    prompt=f'Based on the following feedbacks, provide an overall feedback for the essay:\n\nLanguage Feedback: {state["language_feedback"]}\n\nAnalysis Feedback: {state["analysis_feedback"]}\n\nClarity Feedback: {state["clarity_feedback"]}'
    output=model.invoke(prompt)

    #avg calculation score
    avg_score=sum(state['individual_scores'])/len(state['individual_scores'])

    return {'overall_feedback':output.content,'average_score':avg_score}

In [44]:
graph=StateGraph(UPSCState)

graph.add_node('evaluate_language',evaluate_language)
graph.add_node('evaluate_analysis',evaluate_analysis)
graph.add_node('evaluate_thought',evaluate_thought)
graph.add_node('final_evaluation',final_evaluation)

graph.add_edge(START,'evaluate_language')
graph.add_edge(START,'evaluate_analysis')
graph.add_edge(START,'evaluate_thought')
graph.add_edge('evaluate_language','final_evaluation')
graph.add_edge('evaluate_analysis','final_evaluation')
graph.add_edge('evaluate_thought','final_evaluation')
graph.add_edge('final_evaluation',END)

workflow=graph.compile()

In [47]:
initial_state={
    'essay':essay
}   
#workflow.invoke(initial_state) gives the all the feedback in the dictionary
workflow.invoke(initial_state)

"""{'essay': '" Black holes are among the most fascinating and enigmatic phenomena in the universe. Predicted by Einstein’s theory of general relativity and confirmed through modern observations, black holes challenge our understanding of space, time, and the fundamental laws of physics. Though invisible by nature, their powerful influence on surrounding matter makes them some of the most studied objects in modern astrophysics.\n\nWhat Is a Black Hole?\n\nA black hole is a region of space where gravity is so strong that nothing—not even light—can escape from it. This immense gravitational pull arises when a large amount of mass is compressed into an extremely small area. At the center of a black hole lies the singularity, a point where density becomes infinite and the known laws of physics break down. Surrounding the singularity is the event horizon, the boundary beyond which escape is impossible.\n\nHow Do Black Holes Form?\n\nMost black holes form from the remnants of massive stars. When such a star exhausts its nuclear fuel, it can no longer support itself against gravity and collapses inward, triggering a supernova explosion. If the remaining core is sufficiently massive, it collapses completely, forming a black hole. In addition to these stellar-mass black holes, scientists have discovered supermassive black holes—millions or even billions of times more massive than the Sun—at the centers of most galaxies.',
 'language_feedback': "The essay provides a clear, informative overview of black holes, employing concise language and well-structured sections that facilitate understanding. The introduction effectively sets the context and captures the reader's interest. Definitions and explanations regarding black holes and their formation are precise and scientifically accurate. However, the essay's flow could be improved with smoother transitions between the sections. Additionally, while the technical terms are generally well-utilized, some basic definitions (like what a supernova is) could enhance accessibility for readers less familiar with astrophysics terminology. Overall, the language is formal and objective, fitting for the topic, yet maintaining reader engagement could benefit from varied sentence structures and a touch of narrative element.",
 'analysis_feedback': "The essay provides a solid introduction to black holes, defining what they are and offering a brief explanation of their formation. The explanation of the singularity and event horizon is clear and accurate, which demonstrates a good understanding of the scientific concepts involved. However, the analysis remains rather surface-level and doesn't delve into deeper implications or current research surrounding black holes, such as their role in galaxy formation, Hawking radiation, or theories regarding quantum gravity and the information paradox. Including these topics would have enhanced the depth of the analysis significantly. Furthermore, the essay could benefit from discussing the philosophical and theoretical implications of black holes on our understanding of the universe. Overall, while the foundational information is solid, the lack of deeper exploration limits the essay's analytical depth.",
 'clarity_feedback': 'The essay presents a clear and structured explanation of black holes, demonstrating an understanding of the topic. The introduction effectively sets the stage by highlighting the significance of black holes in astrophysics. The subsequent sections—defining black holes and explaining their formation—are well-organized and easy to follow. Technical terms are introduced logically and explained sufficiently for readers who might not be familiar with them. \n\nHowever, while the clarity of the explanation is strong, the essay could benefit from some expansion on contextual elements, such as the implications of black holes in the broader understanding of the universe, which could enrich the discussion. Additionally, including some real-world examples or recent discoveries related to black holes could engage readers further.\n\nOverall, the essay maintains a solid clarity of thought throughout. \n',
 'overall_feedback': 'Overall Feedback:\n\nThe essay on black holes provides a clear and structured overview, demonstrating a solid understanding of fundamental concepts in astrophysics. The introduction effectively captures the reader\'s interest and establishes the significance of black holes within the field. Definitions and explanations related to black holes, including the singularity and event horizon, are precise and scientifically accurate, making the foundational information accessible to readers.\n\nHowever, while the clarity and organization of the essay are commendable, there are areas for improvement that could enhance the overall depth and engagement of the piece. The transitions between sections could be smoother to create a more cohesive flow, and varying sentence structures could help maintain reader interest. Additionally, introducing basic definitions of key terms, such as "supernova," could further enhance accessibility for audiences less familiar with astrophysical language.\n\nThe analysis, while solid, remains somewhat surface-level. Expanding the discussion to include deeper implications of black holes—such as their role in galaxy formation, current research on Hawking radiation, or the philosophical questions surrounding them—would significantly enrich the content. By including these elements, the essay could offer a more profound exploration of black holes and their significance in understanding the universe.\n\nLastly, incorporating real-world examples or recent discoveries related to black holes could engage readers more effectively and provide context for the information presented. While the essay holds a firm foundation in clarity and structure, enhancing its analytical depth and contextual richness would elevate its overall impact.',
 'individual_scores': [6, 8, 8],
 'average_score': 7.333333333333333}"""

final_state=workflow.invoke(initial_state)

print(f"overall_feedback: {final_state['overall_feedback']}")
print(f"Average Score: {final_state['average_score']}")


overall_feedback: **Overall Feedback:**

The essay provides a foundational introduction to the topic of black holes, characterized by an engaging academic tone and an organized structure. It successfully defines key concepts such as "singularity" and "event horizon," allowing readers with varied backgrounds to grasp the essential ideas of black holes and their formation. The clarity of explanations and logical flow contributes positively to the reader’s understanding.

However, while the essay effectively outlines the basic characteristics and significance of black holes in astrophysics, it falls short in depth and breadth of analysis. Important aspects, such as the implications of black holes on theories of gravity, time dilation, or their role in our understanding of quantum mechanics, are not explored. Furthermore, the absence of citations or references to relevant scientific studies limits its overall credibility. Adding discussions on observational techniques and current controver